# YOLOv5 Face Mask Detection 🧪
This notebook walks through loading a trained YOLOv5 model and testing it on images.

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET

from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import torch
import glob

In [ ]:
def download_dataset(kaggle_json_path, download_path="../data"):
    os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(kaggle_json_path)
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files("andrewmvd/face-mask-detection", path=download_path, unzip=True)
    print("✅ Dataset downloaded and extracted to", download_path)

In [ ]:
def organize_dataset(
    annotations_dir='../data/annotations',
    images_dir='../data/images',
    output_dir='../dataset'
):
    labels_map = {
        'with_mask': 'with_mask',
        'without_mask': 'without_mask',
        'mask_weared_incorrect': 'mask_weared_incorrect'
    }

    # Create a class folder if it doesn't already exist
    for label in labels_map.values():
        os.makedirs(os.path.join(output_dir, label), exist_ok=True)

    # Read the XML file and move the images to the class folder as labeled
    for xml_file in os.listdir(annotations_dir):
        if not xml_file.endswith('.xml'):
            continue

        xml_path = os.path.join(annotations_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        filename = root.find('filename').text
        label = root.find('object').find('name').text

        if label in labels_map:
            src_image_path = os.path.join(images_dir, filename)
            dst_image_path = os.path.join(output_dir, labels_map[label], filename)

            if os.path.exists(src_image_path):
                shutil.copy(src_image_path, dst_image_path)

    print("✅ The dataset has been moved to the per-class folder in the:", output_dir)

In [ ]:
def create_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

    # Freeze base model
    base_model.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

In [ ]:
def plot_training(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title('Loss')
    plt.show()

def evaluate_model(model, val_gen):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_gen.classes
    class_labels = list(val_gen.class_indices.keys())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=val_gen.class_indices, yticklabels=val_gen.class_indices)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    print("Confusion Matrix:")
    print(cm)
    
def evaluate_model_normalized(model, val_gen):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_gen.classes
    class_labels = list(val_gen.class_indices.keys())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("Normalized Confusion Matrix")
    print("Confusion Matrix:")
    print(cm)

In [ ]:
def save_predictions(y_true, y_pred, class_indices, file_path="../visualizations/predictions.csv"):
    class_labels = list(class_indices.keys())
    inverse_map = {v: k for k, v in class_indices.items()}

    df = pd.DataFrame({
        'Actual': [inverse_map[i] for i in y_true],
        'Predicted': [inverse_map[i] for i in y_pred]
    })

    df.to_csv(file_path, index=False)
    print(f"📄 Saved prediction results to {file_path}")

In [ ]:
def predict_single_image(image_path, model, class_indices, img_size=(224, 224)):
    img = tf.keras.utils.load_img(image_path, target_size=img_size)
    img_array = tf.keras.utils.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    class_labels = list(class_indices.keys())

    print(f"📌 Image: {image_path}")
    print(f"Predicted: {class_labels[predicted_class]}")

In [ ]:
# Download dataset
kaggle_json_path = "../kaggle.json"  # Adjust path if needed
if not os.path.exists("../data/annotations"):  # simple check if already extracted
    download_dataset(kaggle_json_path)

In [ ]:
# Preprocessing
organize_dataset()

img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)


train_gen = datagen.flow_from_directory(
    '../dataset/',  # fixed path
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    '../dataset/',  # fixed path
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    "../dataset/",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Model
model = create_model(input_shape=img_size + (3,), num_classes=3)

# Recompile
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Training
if not os.path.exists("../models"):
    os.makedirs("../models")
    
checkpoint_cb = ModelCheckpoint("../models/best_mask_detector.h5", save_best_only=True, monitor='val_accuracy', mode='max')
earlystop_cb = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=3,
    callbacks=[checkpoint_cb, earlystop_cb]
)

model.save("../models/mask_detector.h5")

In [ ]:
# Evaluation
if not os.path.exists("../visualizations"):
    os.makedirs("../visualizations")

plot_training(history)
evaluate_model(model, test_gen)
evaluate_model_normalized(model, test_gen)

In [ ]:
# Take y_true and y_pred from prediction
test_gen.reset()
preds = model.predict(test_gen, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = test_gen.classes

save_predictions(y_true, y_pred, val_gen.class_indices)

In [ ]:
loss, acc = model.evaluate(test_gen)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")

In [ ]:
predict_single_image("../test/sample_test.png", model, train_gen.class_indices)

In [ ]:
# === YOLOv5 TESTING ===

print("\n🔍 Running YOLOv5 detection on sample image...")

# Load YOLOv5 pretrained model
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/yolov5/yolov5s.pt')

# Load a sample image from the dataset (with_mask as example)
sample_images = glob.glob("../dataset/with_mask/*.jpg") + glob.glob("../dataset/with_mask/*.png")
if sample_images:
    test_img = sample_images[0]
    print("🖼️ Image path:", test_img)
    results = yolo_model(test_img)
    results.print()
    results.show()
else:
    print("❌ No image found in ../dataset/with_mask/")

In [ ]:
import random

for category in ['with_mask', 'without_mask', 'mask_weared_incorrect']:
    sample_images = glob.glob(f"../dataset/{category}/*.jpg") + glob.glob(f"../dataset/{category}/*.png")
    if sample_images:
        test_img = random.choice(sample_images)
        print(f"\n🖼️ Testing image from '{category}': {test_img}")
        results = yolo_model(test_img)
        results.print()
        results.show()
    else:
        print(f"❌ No image found in ../dataset/{category}/")